# Credit Score model - EDA

**Objective**: In this notebook we make the EDA analisys of the database labeled in 1.data_labeling.ipynb

Here we have:

1. Data loading and Initial Exploration.
2. First needed feature engineering based on the purpose of the modeling.
3. Second feature engeneering based on business criteria
4. EDA over remaining variables.
5. Feature selection.
6. Final databases storing.


# 1. Data loading and Initial Exploration.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path

# Auxiliary functions made for this project
from utils.functions import (
    DataProfile,
    FeatureEngineering,
    StabilityMetrics,
    WOEAnalysis
)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
sns.set_palette("husl")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = Path("data/processed")

train = pd.read_parquet(data_dir / "train.parquet", engine="fastparquet")
valid = pd.read_parquet(data_dir / "valid.parquet", engine="fastparquet")
test = pd.read_parquet(data_dir / "test.parquet", engine="fastparquet")

print("Dataset Shapes:")
print(f"Train: {train.shape}")
print(f"Valid: {valid.shape}")
print(f"Test:  {test.shape}")

print("\nDate Ranges:")
print(f"Train: {train['issue_d'].min()} → {train['issue_d'].max()}")
print(f"Valid: {valid['issue_d'].min()} → {valid['issue_d'].max()}")
print(f"Test:  {test['issue_d'].min()} → {test['issue_d'].max()}")

Dataset Shapes:
Train: (1621480, 145)
Valid: (462928, 145)
Test:  (190890, 145)

Date Ranges:
Train: 2013-01-01 00:00:00 → 2017-12-01 00:00:00
Valid: 2018-01-01 00:00:00 → 2018-12-01 00:00:00
Test:  2019-01-01 00:00:00 → 2019-12-01 00:00:00


# 2. First needed feature engineering based on the purpose of the modeling.

Given that we want to buld an origination score, variables related with the post-origination must be discarted, based on descriptions we have:

- loan_status
- total_pymnt
- total_pymnt_inv
- total_rec_prncp
- total_rec_int
- total_rec_late_fee
- out_prncp
- out_prncp_inv
- recoveries
- collection_recovery_fee
- last_pymnt_d
- last_pymnt_amnt
- next_pymnt_d
- last_credit_pull_d
- last_fico_range_low
- last_fico_range_high
- pymnt_plan
- deferral_term
- orig_projected_additional_accrued_interest

Aditionally we have prefixes that indicate the feature is a post-origination feature.

Perhaps, features related to the pricing must to be droped to avoid redundancy in modeling. Based on descriptions we have:

- int_rate
- grade
- sub_grade
- initial_list_status
- funded_amnt
- funded_amnt_inv


In [3]:
fe = FeatureEngineering()

PRICING_ARTIFACTS = [
    "int_rate",
    "grade",
    "sub_grade",
    "initial_list_status",
    "funded_amnt",
    "funded_amnt_inv",
]

POST_ORIG_BASE = [
    "loan_status",
    "total_pymnt",
    "total_pymnt_inv",
    "total_rec_prncp",
    "total_rec_int",
    "total_rec_late_fee",
    "out_prncp",
    "out_prncp_inv",
    "recoveries",
    "collection_recovery_fee",
    "last_pymnt_d",
    "last_pymnt_amnt",
    "next_pymnt_d",
    "last_credit_pull_d",
    "last_fico_range_low",
    "last_fico_range_high",
    "pymnt_plan",
    "deferral_term",
    "orig_projected_additional_accrued_interest",
]

FAMILY_PREFIXES = ("hardship_", "settlement_", "debt_settlement_flag")
family_cols = [c for c in train.columns if c.startswith(FAMILY_PREFIXES)]

drop_cols = set(PRICING_ARTIFACTS) | set(POST_ORIG_BASE) | set(family_cols)

train_filtered = fe.make_pre_offer_features(
    train, keep_term=False, keep_target=True, drop_cols=drop_cols
)
valid_filtered = fe.make_pre_offer_features(
    valid, keep_term=False, keep_target=True, drop_cols=drop_cols
)
test_filtered = fe.make_pre_offer_features(
    test, keep_term=False, keep_target=True, drop_cols=drop_cols
)

print(f"Kept {len(train_filtered.columns)} features + target")

Kept 101 features + target


# 3. Second feature engeneering based on business criteria

In [4]:
profiler = DataProfile()
profile = profiler.quick_profile(train_filtered, exclude=["target"])
print("\nData Quality Summary:")
display(profile.head(55))


Data Quality Summary:


,feature,dtype,missing_pct,n_unique,min,median,max,top_category,top_pct
0,sec_app_revol_util,float64,97.77,1146,0.0,62.40,1.825000e+02,NaN,NaN
1,revol_bal_joint,float64,97.74,27895,0.0,25323.00,3.571350e+05,NaN,NaN
2,sec_app_chargeoff_within_12_mths,float64,97.74,19,0.0,0.00,2.100000e+01,NaN,NaN
3,sec_app_collections_12_mths_ex_med,float64,97.74,14,0.0,0.00,1.600000e+01,NaN,NaN
4,sec_app_earliest_cr_line,object,97.74,598,NaN,NaN,NaN,Aug-2006,0.91
5,sec_app_fico_range_high,float64,97.74,61,544.0,669.00,8.500000e+02,NaN,NaN
6,sec_app_fico_range_low,float64,97.74,61,540.0,665.00,8.450000e+02,NaN,NaN
7,sec_app_inq_last_6mths,float64,97.74,7,0.0,0.00,6.000000e+00,NaN,NaN
8,sec_app_mort_acc,float64,97.74,18,0.0,1.00,1.800000e+01,NaN,NaN
9,sec_app_num_rev_accts,float64,97.74,74,0.0,11.00,9.600000e+01,NaN,NaN


`application_type` feature shows that 97% of loans are `individual`. Given the low share of joint loans, features related to the second applicant are going to be droped. Those that start with `sec_app_` and those that end with `joint`

In [5]:
sec_app_cols = [c for c in train_filtered.columns if c.startswith("sec_app_")]

JOINT_FEATURES = {
    "annual_inc_joint",
    "dti_joint",
    "verification_status_joint",
    "revol_bal_joint",
    "application_type",
}

UTILITY_TEXT = ["url", "policy_code", "member_id", "id", "Unnamed: 0", "emp_title", "title"]

drop_cols = set(UTILITY_TEXT) | set(sec_app_cols) | JOINT_FEATURES

train_filtered = fe.make_pre_offer_features(
    train_filtered, keep_term=False, keep_target=True, drop_cols=drop_cols
)
valid_filtered = fe.make_pre_offer_features(
    valid_filtered, keep_term=False, keep_target=True, drop_cols=drop_cols
)
test_filtered = fe.make_pre_offer_features(
    test_filtered, keep_term=False, keep_target=True, drop_cols=drop_cols
)

print(f"Kept {len(train_filtered.columns)} features + target")

Kept 79 features + target


#### 3.1. Feature Engineering over `emp_length`, `revol_util` and `fico_range` features

Some transformations are made over those variables to analize them better, this is:

1. emp_length: Transform it into a monotonic variable.
2. revol_util: is a string because percentages have `%` at the end. Transform it into a numerical variable.
3. fico_range: Median of fico_range variables (upper and lower) is taken to construct a new feature.

In [6]:
train_eng = fe.engineer_features(train_filtered)
valid_eng = fe.engineer_features(valid_filtered)
test_eng = fe.engineer_features(test_filtered)

train_eng.drop(columns=["emp_length", "revol_util", "fico_range_low", "fico_range_high"], inplace=True)
valid_eng.drop(columns=["emp_length", "revol_util", "fico_range_low", "fico_range_high"], inplace=True)
test_eng.drop(columns=["emp_length", "revol_util", "fico_range_low", "fico_range_high"], inplace=True)

### 3.2. some imputations based on bussiness criteria

NanN values in buro variables like `open_acc_6m` could mean that there is no info for the variable, wich make sence. Next we impute some NaN values with common used values for this cases:

- 999: for variables that starts with `mths_since_` and these 2 features `num_tl_120dpd_2m`  and `mo_sin_old_il_acct`
- 0: for counts

In [7]:
count_cols = [
    "il_util",
    "all_util",
    "bc_util",
    "percent_bc_gt_75",
    "bc_open_to_buy",
    "open_acc_6m",
    "open_il_12m",
    "open_il_24m",
    "open_rv_12m",
    "open_rv_24m",
    "inq_fi",
    "inq_last_12m",
    "total_bal_il",
    "total_cu_tl",
    "max_bal_bc",
    "open_act_il",
]

no_rec_cols = ["num_tl_120dpd_2m", "mo_sin_old_il_acct"]

train_clean = fe.handle_missing_values(
    train_eng, prefix="mths_since_", count_cols=count_cols, no_rec_cols=no_rec_cols
)
valid_clean = fe.handle_missing_values(
    valid_eng, prefix="mths_since_", count_cols=count_cols, no_rec_cols=no_rec_cols
)
test_clean = fe.handle_missing_values(
    test_eng, prefix="mths_since_", count_cols=count_cols, no_rec_cols=no_rec_cols
)

### 3.3 Default variable mapping

In [8]:
target_mapping = {"non_default": 0, "default": 1}

train_clean["target"] = train_clean["target"].map(target_mapping).astype("int8")
valid_clean["target"] = valid_clean["target"].map(target_mapping).astype("int8")
test_clean["target"] = test_clean["target"].map(target_mapping).astype("int8")

print("\nTarget Distribution:")
print(test_clean["target"].value_counts())
print(f"\nDefault Rate: {test_clean['target'].mean():.2%}")


Target Distribution:
target
0    177325
1     13565
Name: count, dtype: int64

Default Rate: 7.11%


# 4. Exploratory Data Analysis (EDA)

Quick view of variables profile.

In [9]:
train_final = train_clean.copy()
valid_final = valid_clean.copy()
test_final = test_clean.copy()

profile = profiler.quick_profile(train_final, exclude=["target"])
print("\nData Quality Summary:")
display(profile.head(20))


Data Quality Summary:


,feature,dtype,missing_pct,n_unique,min,median,max,top_category,top_pct
0,emp_length_yrs,float32,6.09,11,0.5,6.000000,1.000000e+01,NaN,NaN
1,revol_util_pct,float32,0.06,1314,0.0,52.400002,8.923000e+02,NaN,NaN
2,dti,float64,0.03,8169,-1.0,18.100000,9.990000e+02,NaN,NaN
3,pct_tl_nvr_dlq,float64,0.01,672,0.0,97.700000,1.000000e+02,NaN,NaN
4,acc_now_delinq,float64,0.00,9,0.0,0.000000,1.400000e+01,NaN,NaN
5,acc_open_past_24mths,float64,0.00,57,0.0,4.000000,6.400000e+01,NaN,NaN
6,addr_state,object,0.00,51,NaN,NaN,NaN,CA,13.73
7,all_util,float64,0.00,183,0.0,26.000000,2.110000e+02,NaN,NaN
8,annual_inc,float64,0.00,72661,0.0,65000.000000,1.100000e+08,NaN,NaN
9,avg_cur_bal,float64,0.00,80953,0.0,7380.000000,9.580840e+05,NaN,NaN


## 4.1. Creation of useful ratios

Ratios are very useful for scorecad modelling as they measure variables relative to others. Feature related to income are very informative. So we create the respective ratios "to_income" and "to_limit", and drop the original variables.

In [10]:
for df in (train_final, valid_final, test_final):
    df["loan_to_income"] = df["loan_amnt"] / df["annual_inc"]
    df["install_to_income"] = df["installment"] / (df["annual_inc"] / 12)
    df["util_to_limit"] = df["tot_cur_bal"] / df["tot_hi_cred_lim"]
    df["balance_to_income"] = df["tot_cur_bal"] / df["annual_inc"]
    df.drop(
        columns=["loan_amnt", "installment", "tot_cur_bal", "annual_inc", "tot_hi_cred_lim"],
        inplace=True
    )

### Binning of numerical important varioables

From a off iteration it was discovered that binning some numerical variables and some ratios could help to improve model stability and performance of the model. Here we bin some of the most important numerical variables based on business criteria and based on our offline analysis of WOE discrimination per bin.

In [11]:
num_bins = {
    "loan_to_income": [0, 0.04, 0.08, 0.12, 0.16, 0.22, 0.30, np.inf],
    "inq_last_6mths": [0, 1, 2, np.inf],
    "avg_cur_bal": [0, 11000, 15000, 25000, 30000, np.inf],
    "install_to_income": [-0.00001, 0.02, 0.04, 0.07, 0.1, np.inf],
    "util_to_limit": [-0.00001, 0.5, 1, np.inf],
    "balance_to_income": [-0.00001, 0.4, np.inf],
    "delinq_2yrs": [-0.001, 1, np.inf],
    "inq_last_12m": [-0.001, 1, 4, np.inf],
    "mths_since_last_delinq": [-0.001, 900],
    "revol_util_pct": [-0.00001, 30, 60, 90, np.inf],
}

dfs = [train_final, valid_final, test_final]

for col, edges in num_bins.items():
    for df in dfs:
        if col in df.columns:
            x = pd.to_numeric(df[col], errors="coerce")
            b = pd.cut(x, bins=np.array(edges, dtype=float), include_lowest=True)
            lab = b.astype(str).where(b.notna(), "no_info")
            lab = lab.str.replace(r"[^A-Za-z0-9]+", "_", regex=True).str.strip("_")
            df[f"{col}_bin"] = pd.Categorical(lab)

for df in dfs:
    df.drop(columns=[c for c in num_bins if c in df.columns], inplace=True)

## 4.2 WOE segmentation analisys
Here an example of how variables discriminate by bin based on woe analisys:

In [12]:
woe_analyzer = WOEAnalysis()

iv_overview = woe_analyzer.show_woe_for_columns(
    train_final,
    target="target",
    num_cols=[f"{name}_bin" for name in num_bins.keys()],
    show_tables=True,
)

loan_to_income_bin


,Agrupacion,Total,G,B,%_bad_rate,WOE,IV_bin
0,0_001_0_04,44185,39757,4428,10.0%,0.661478,0.009542
1,0_04_0_08,136100,119725,16375,12.0%,0.456082,0.014995
2,0_08_0_12,193414,168000,25414,13.1%,0.355304,0.013384
3,0_12_0_16,219308,187469,31839,14.5%,0.239562,0.007174
4,0_16_0_22,314338,262613,51725,16.5%,0.091381,0.001571
5,0_22_0_3,337655,271191,66464,19.7%,-0.127197,0.003508
6,0_3_inf,376480,284908,91572,24.3%,-0.398319,0.041637


inq_last_6mths_bin


,Agrupacion,Total,G,B,%_bad_rate,WOE,IV_bin
0,0_001_1_0,1401878,1166300,235578,16.8%,0.066190,0.003707
1,1_0_2_0,147602,114100,33502,22.7%,-0.307890,0.009497
2,2_0_inf,71999,53262,18737,26.0%,-0.488637,0.012296
3,no_info,1,1,0,0.0%,-0.287929,0.000000


avg_cur_bal_bin


...

,Agrupacion,Total,G,B,%_bad_rate,WOE,IV_bin
0,0_002_1_0,1202334,998504,203830,17.0%,0.055612,0.002252
1,1_0_4_0,310434,251344,59090,19.0%,-0.085599,0.001442
2,4_0_inf,108712,83815,24897,22.9%,-0.319495,0.007558


mths_since_last_delinq_bin


,Agrupacion,Total,G,B,%_bad_rate,WOE,IV_bin
0,0_002_900_0,823324,672882,150442,18.3%,-0.035367,0.000642
1,no_info,798156,660781,137375,17.2%,0.037348,0.000678


revol_util_pct_bin


,Agrupacion,Total,G,B,%_bad_rate,WOE,IV_bin
0,0_00101_30_0,327969,279338,48631,14.8%,0.214802,0.008697
1,30_0_60_0,660394,542749,117645,17.8%,-0.004384,0.000008
2,60_0_90_0,534720,432563,102157,19.1%,-0.090143,0.002758
3,90_0_inf,97374,78151,19223,19.7%,-0.130824,0.001071
4,no_info,1023,862,161,15.7%,0.144491,0.000013


## 4.3 Information value (IV)

Next, is calculated the information value (IV) for all the features. It is well known that variables with an IV over 0.2 could have a high power of prediction in a scorecard model. Fore numerical variables, here are very important the bins we've already built, but for some others bins are constructed based on percentiles of distribution.


In [13]:
iv_overview = woe_analyzer.show_woe_for_columns(
    train_final,
    target="target",
    num_cols=train_final.columns.difference(["target"]).tolist(),
    cat_cols=[],
)
display(iv_overview.style.format({"iv_total": "{:.4f}"}))

,feature,iv_total,type
14,fico_mid,0.1122,numeric
21,loan_to_income_bin,0.0918,categorical
20,install_to_income_bin,0.0857,categorical
1,acc_open_past_24mths,0.0836,numeric
49,num_tl_op_past_12m,0.0590,numeric
74,verification_status,0.0559,categorical
6,bc_open_to_buy,0.0509,numeric
12,dti,0.0479,numeric
69,total_bc_limit,0.0432,numeric
4,avg_cur_bal_bin,0.0422,categorical


## 4.4 Population Stability Index

To improve the stability of the model over time, it is better to choose variables that are stable over time in terms of PSI. Here we calculate the PSI per variable for train vs validation. PSI >0.25 could indicate that the feature is very unstable, so it can be droped if it's information value is not important.

In [14]:
stability = StabilityMetrics()

num_cols = [
    c for c in train_final.columns if c != "target" and pd.api.types.is_numeric_dtype(train_final[c])
]
cat_cols = [
    c for c in train_final.columns if c != "target" and not pd.api.types.is_numeric_dtype(train_final[c])
]

psi_results = []
for col in num_cols:
    psi_val = stability.calculate_psi_numeric(train_final[col], valid_final[col])
    psi_results.append({"feature": col, "psi": psi_val})

for col in cat_cols:
    psi_val = stability.calculate_psi_categorical(train_final[col], valid_final[col])
    psi_results.append({"feature": col, "psi": psi_val})

psi_df = pd.DataFrame(psi_results).sort_values("psi", ascending=False)

print("\nPopulation Stability (PSI - Train vs Valid):")
display(psi_df.head(15))


Population Stability (PSI - Train vs Valid):


,feature,psi
14,mths_since_rcnt_il,1.132497
19,max_bal_bc,1.008996
20,all_util,0.898630
15,total_bal_il,0.445214
16,il_util,0.402228
11,open_act_il,0.315804
18,open_rv_24m,0.268341
13,open_il_24m,0.152401
73,inq_last_12m_bin,0.150994
22,inq_fi,0.127562


## 5. Feature Selection

Next we drop features with `IV< 0.02`, `PSI>0.25`and we add an additional filter for strong correlated features. To reduce noise, features with `correlation>0.85` are analized and one of them is dropped. 

In [15]:
IV_MIN = 0.02
PSI_MAX = 0.25
CORR_THRESHOLD = 0.85

iv_pass = set(iv_overview.loc[iv_overview.iv_total >= IV_MIN, "feature"])
psi_pass = set(psi_df.loc[psi_df.psi <= PSI_MAX, "feature"])
base_features = iv_pass & psi_pass

engineered = {"fico_mid", "emp_length_yrs"}
base_features |= engineered

unstable_features = {
    "max_bal_bc",
    "all_util",
    "total_bal_il",
    "il_util",
    "open_act_il",
    "mths_since_last_record",
}
base_features -= unstable_features

print(f"Features after IV/PSI filtering: {len(base_features)}")

Features after IV/PSI filtering: 23


In [16]:
numeric_candidates = [
    c for c in base_features if c in train_final.columns and pd.api.types.is_numeric_dtype(train_final[c])
]

corr_matrix = train_final[numeric_candidates].corr(method="spearman").abs()
iv_lookup = iv_overview.set_index("feature").iv_total.to_dict()

selected = []
dropped = set()

for feat in sorted(numeric_candidates, key=lambda x: -iv_lookup.get(x, 0)):
    if feat in dropped:
        continue
    selected.append(feat)
    highly_corr = corr_matrix.index[
        (corr_matrix[feat] >= CORR_THRESHOLD) & (corr_matrix.index != feat)
    ].tolist()
    dropped.update(highly_corr)

final_features = sorted((set(selected) | (base_features - set(numeric_candidates))) - dropped)

FINAL_FEATURE_SET = final_features


print(f"Number of final features: {len(final_features)}")
print(f"\nSelected Features: {final_features}")

Number of final features: 21

Selected Features: ['acc_open_past_24mths', 'avg_cur_bal_bin', 'bc_open_to_buy', 'dti', 'emp_length_yrs', 'fico_mid', 'home_ownership', 'inq_last_6mths_bin', 'install_to_income_bin', 'loan_to_income_bin', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_rev_tl', 'num_tl_op_past_12m', 'percent_bc_gt_75', 'total_bc_limit', 'verification_status']


In [17]:
psi_final = psi_df[psi_df.feature.isin(FINAL_FEATURE_SET)].sort_values("psi", ascending=False)
print("\nPSI for Final Features:")
display(psi_final)


PSI for Final Features:


,feature,psi
53,percent_bc_gt_75,0.121805
25,bc_open_to_buy,0.107556
60,fico_mid,0.103101
62,verification_status,0.068883
57,total_bc_limit,0.043779
30,mo_sin_old_rev_tl_op,0.040733
67,inq_last_6mths_bin,0.025634
33,mort_acc,0.024842
40,num_actv_rev_tl,0.023098
0,dti,0.019787


# 6. Final Feature Storing

In [18]:
X_train = train_final[FINAL_FEATURE_SET].copy()
y_train = train_final["target"].values

X_valid = valid_final[FINAL_FEATURE_SET].copy()
y_valid = valid_final["target"].values

X_test = test_final[FINAL_FEATURE_SET].copy()
y_test = test_final["target"].values


In [19]:
X_train.to_parquet("data/eda/X_train.parquet", index=False)
X_valid.to_parquet("data/eda/X_valid.parquet", index=False)
X_test.to_parquet("data/eda/X_test.parquet", index=False)

pd.DataFrame({"target": y_train}).to_parquet("data/eda/y_train.parquet", index=False)
pd.DataFrame({"target": y_valid}).to_parquet("data/eda/y_valid.parquet", index=False)
pd.DataFrame({"target": y_test}).to_parquet("data/eda/y_test.parquet", index=False)

np.save("data/eda/y_train.npy", y_train)
np.save("data/eda/y_valid.npy", y_valid)
np.save("data/eda/y_test.npy", y_test)